# Local LLMs with LiteLLM & Ollama

In this notebook we'll create two agents, Joe and Cathy who like to tell jokes to each other. The agents will use locally running LLMs.

Follow the guide at https://microsoft.github.io/autogen/docs/topics/non-openai-models/local-litellm-ollama/ to understand how to install LiteLLM and Ollama.

We encourage going through the link, but if you're in a hurry and using Linux, run these:  
  
```
curl -fsSL https://ollama.com/install.sh | sh

ollama pull llama3.2:1b

pip install 'litellm[proxy]'
litellm --model ollama/llama3.2:1b
```  

This will run the proxy server and it will be available at 'http://0.0.0.0:4000/'.

To get started, let's import some classes.

In [1]:
%%capture
# capture magic suppresses install output
!poetry add autogen_core autogen_ext openai tiktoken 'litellm[proxy]'

In [2]:
!if [[ "$(ollama list llama3.2:1b | grep -v "NAME" | grep llama)" == "" ]]; then echo "Download model w/ 'ollama pull llama3.2:1b'"; fi

In [6]:
!litellm --model ollama/llama3.2:1b

/Users/josephhunt/Library/Caches/pypoetry/virtualenvs/autogen-notebooks-iuI5xiTZ-py3.10/lib/python3.10/site-packages/pydantic/_internal/_config.py:345: UserWarning: Valid config keys have changed in V2:
* 'fields' has been removed
  warnings.warn(message, UserWarning)
INFO:     Started server process [7835]
INFO:     Waiting for application startup.

#------------------------------------------------------------#
#                                                            #
#            'This product would be better if...'             #
#        https://github.com/BerriAI/litellm/issues/new        #
#                                                            #
#------------------------------------------------------------#

 Thank you for using LiteLLM! - Krrish & Ishaan



Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new


INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:4000 (Press CTRL+C to quit)
^C
INFO:     Shutting down
IN

In [7]:
# Setup logging to new/overwritten logfile with loglevel
import logging
from pathlib import Path

Path("./logs").mkdir(parents=True, exist_ok=True)
logfile = f"./logs/local-llms.log"
logfmode = 'w'                # w = overwrite, a = append
# Log levels: NOTSET DEBUG INFO WARN ERROR CRITICAL
loglevel = logging.DEBUG
# Cuidado! DEBUG will leak secrets!
logging.basicConfig(filename=logfile, encoding='utf-8', level=loglevel, filemode=logfmode)

In [8]:
from dataclasses import dataclass

from autogen_core import (
    AgentId,
    DefaultTopicId,
    MessageContext,
    RoutedAgent,
    SingleThreadedAgentRuntime,
    default_subscription,
    message_handler,
)
from autogen_core.model_context import BufferedChatCompletionContext
from autogen_core.models import (
    AssistantMessage,
    ChatCompletionClient,
    SystemMessage,
    UserMessage,
)
from autogen_ext.models.openai import OpenAIChatCompletionClient

Set up out local LLM model client.

In [9]:
def get_model_client() -> OpenAIChatCompletionClient:  # type: ignore
    "Mimic OpenAI API using Local LLM Server."
    return OpenAIChatCompletionClient(
        model="llama3.2:1b",
        api_key="NotRequiredSinceWeAreLocal",
        base_url="http://0.0.0.0:4000",
        model_capabilities={
            "json_output": False,
            "vision": False,
            "function_calling": True,
        },
    )

Define a simple message class

In [10]:
@dataclass
class Message:
    content: str

Now, the Agent.

We define the role of the Agent using the `SystemMessage` and set up a condition for termination.

In [11]:
@default_subscription
class Assistant(RoutedAgent):
    def __init__(self, name: str, model_client: ChatCompletionClient) -> None:
        super().__init__("An assistant agent.")
        self._model_client = model_client
        self.name = name
        self.count = 0
        self._system_messages = [
            SystemMessage(
                content=f"Your name is {name} and you are a part of a duo of comedians."
                "You laugh when you find the joke funny, else reply 'I need to go now'.",
            )
        ]
        self._model_context = BufferedChatCompletionContext(buffer_size=5)

    @message_handler
    async def handle_message(self, message: Message, ctx: MessageContext) -> None:
        self.count += 1
        await self._model_context.add_message(UserMessage(content=message.content, source="user"))
        result = await self._model_client.create(self._system_messages + await self._model_context.get_messages())

        print(f"\n{self.name}: {message.content}")

        if "I need to go".lower() in message.content.lower() or self.count > 2:
            return

        await self._model_context.add_message(AssistantMessage(content=result.content, source="assistant"))  # type: ignore
        await self.publish_message(Message(content=result.content), DefaultTopicId())  # type: ignore

Set up the agents.

In [12]:
runtime = SingleThreadedAgentRuntime()

cathy = await Assistant.register(
    runtime,
    "cathy",
    lambda: Assistant(name="Cathy", model_client=get_model_client()),
)

joe = await Assistant.register(
    runtime,
    "joe",
    lambda: Assistant(name="Joe", model_client=get_model_client()),
)

Let's run everything!

In [13]:
runtime.start()
await runtime.send_message(
    Message("Joe, tell me a joke."),
    recipient=AgentId(joe, "default"),
    sender=AgentId(cathy, "default"),
)
await runtime.stop_when_idle()

/var/folders/kd/zg06nphx4kn32n3nr75fk3x40000gn/T/ipykernel_7566/172301977.py:20: UserWarning: Resolved model mismatch: llama3.2:1b != ollama/llama3.2:1b. Model mapping may be incorrect.
  result = await self._model_client.create(self._system_messages + await self._model_context.get_messages())



Joe: Joe, tell me a joke.

Cathy: (laughs) Ahahahaha, okay, here's one: Why couldn't the bicycle stand up by itself? (pauses for comedic effect)... Because it was two-tired! (laughs) Get it? Two-tired... I need to go now.


In [14]:
await runtime.stop()

RuntimeError: Runtime is not started